# 02 - Data Cleaning & Feature Engineering

Purpose:
- Clean Netflix Titles dataset: handle missing values, fix data types, parse dates.
- Engineer a few helpful features for analysis and recommender:
  - `primary_country`, `genres_list` / `genres_str`, `date_added_year`, `date_added_month`, `director_primary`, `cast_primary`, `metadata`.
- Save cleaned, smaller CSV at `data/cleaned/netflix_titles_clean.csv`.

In [193]:
# Imports and path constants
import os
from pathlib import Path
import pandas as pd
import numpy as np
from IPython.display import display

# Paths
RAW_CSV = Path(r"C:\Users\Admibn\OneDrive\Desktop\Netflix Project\netflix-analysis-project\data\raw\netflix_titles.csv")
RAW_XLSX = Path(r"C:\Users\Admibn\OneDrive\Desktop\Netflix Project\netflix-analysis-project\data\raw\netflix_titles.xlsx")
CLEAN_DIR = Path(r"C:\Users\Admibn\OneDrive\Desktop\Netflix Project\netflix-analysis-project\data\cleaned")
CLEAN_CSV = CLEAN_DIR / "netflix_titles_clean.csv"

pd.set_option("display.max_columns", 180)
pd.set_option("display.max_colwidth", 200)

In [194]:
# Load dataset
if RAW_CSV.exists():
    df = pd.read_csv(RAW_CSV)
    print("Loaded CSV: ", RAW_CSV)
elif RAW_XLSX.exists():
    df = pd.read_excel(RAW_XLSX)
    print("Loaded Excel: ", RAW_XLSX)
else:
    raise FileNotFoundError("No dataset found in data/raw. Put netflix_titles.csv or netflix_titles.xlsx there.")

print("Shape: ", df.shape)
display(df.head(5))

Loaded CSV:  C:\Users\Admibn\OneDrive\Desktop\Netflix Project\netflix-analysis-project\data\raw\netflix_titles.csv
Shape:  (8807, 12)


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmmaker Kirsten Johnson stages his death in inventive and comical ways to help them both face the inevitable."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thabang Molaba, Dillon Windvogel, Natasha Thahane, Arno Greeff, Xolile Tshabalala, Getmore Sithole, Cindy Mahlangu, Ryle De Morny, Greteli Fincham, Sello Ma...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town teen sets out to prove whether a private-school swimming star is her sister who was abducted at birth."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabiha Akkari, Sofia Lesaffre, Salim Kechiouche, Noureddine Farihi, Geert Van Rampelberg, Bakary Diombera",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Action & Adventure","To protect his family from a powerful drug lord, skilled thief Mehdi and his expert team of robbers are pulled into a violent and deadly turf war."
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down among the incarcerated women at the Orleans Justice Center in New Orleans on this gritty reality series."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam Khan, Ahsaas Channa, Revathi Pillai, Urvi Singh, Arun Kumar",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV Comedies","In a city of coaching centers known to train India���s finest collegiate minds, an earnest but unexceptional student and his friends navigate campus life."


In [195]:
# Schema and missing values
print("Columns: ", list(df.columns))
print("\nDtypes:")
display(df.dtypes)
print("\nMissing values (descending):")
display(df.isnull().sum().sort_values(ascending=False).head(30)) # Display top 30 missing

Columns:  ['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added', 'release_year', 'rating', 'duration', 'listed_in', 'description']

Dtypes:


show_id         object
type            object
title           object
director        object
cast            object
country         object
date_added      object
release_year     int64
rating          object
duration        object
listed_in       object
description     object
dtype: object


Missing values (descending):


director        2634
country          831
cast             825
date_added        10
rating             4
duration           3
show_id            0
type               0
title              0
release_year       0
listed_in          0
description        0
dtype: int64

In [196]:
# Normalize column names and trim whitespace
df.columns = [c.strip() for c in df.columns] # Keep original case but strip whitespace
# Trim whitespace in object columns
for c in df.select_dtypes(include="object").columns: # Only object columns
    df[c] = df[c].astype("string").str.strip() # Use pandas string dtype to allow NaN
print("Column names normalized. Sample:")
df.head(1)

Column names normalized. Sample:


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,<NA>,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmmaker Kirsten Johnson stages his death in inventive and comical ways to help them both face the inevitable."


In [197]:
# Convert date_added to datetime and extract year/month
if 'date_added' in df.columns:
    df['date_added'] = pd.to_datetime(df['date_added'], errors='coerce') # Coerce errors to NaT
    df['date_added_year'] = df['date_added'].dt.year.astype('Int64') # Use Int64 to allow NaN
    df['date_added_month'] = df['date_added'].dt.month.astype('Int64') # Use Int64 to allow NaN
    print("'date_added' parsed. \nRange:", df['date_added'].min(), "to", df['date_added'].max())
else:
    print("No 'date_added' column present.")

'date_added' parsed. 
Range: 2008-01-01 00:00:00 to 2021-09-25 00:00:00


In [198]:
# Primary country
if 'country' in df.columns:
    df['country'] = df['country'].replace('', pd.NA) # Replace empty strings with NaN
    df['primary_country'] = (df['country']
                              .fillna('') # Fill NaN with empty string to avoid errors
                              .str.split(',') # Split by comma to get list of countries
                              .str[0] # Take first country
                              .str.strip() # Strip whitespace
                              .replace('', 'Unknown') # Replace empty strings with 'Unknown'
                              .astype('string')) # Convert to string dtype
    print("Primary country examples:")
    display(df['primary_country'].value_counts().head(10)) # Show top 10 countries
else:
    df['primary_country'] = 'Unknown'
    print("No country column, created primary_country = 'Unknown'")

Primary country examples:


primary_country
United States     3211
India             1008
Unknown            833
United Kingdom     628
Canada             271
Japan              259
France             212
South Korea        211
Spain              181
Mexico             134
Name: count, dtype: Int64

In [199]:
# Parse listed_in -> genres_list and genres_str
if 'listed_in' in df.columns:
    df['listed_in'] = df['listed_in'].fillna('') # Replace NaN with empty string
    df['genres_list'] = df['listed_in'].apply(lambda s: [g.strip() for g in s.split(',') if g.strip()]) # Split and strip
    df['genres_str'] = df['genres_list'].apply(lambda lst: "|".join(lst) if lst else "") # Join with | or empty string
    print("Top genres (exploded):")
    display(pd.Series([g for lst in df['genres_list'] for g in lst]).value_counts().head(15)) # Explode and count top 15
else:
    df['genres_list'] = [[] for _ in range(len(df))] # Empty list for each row
    df['genres_str'] = "" # Empty string
    print("No 'listed_in' column.")

Top genres (exploded):


International Movies        2752
Dramas                      2427
Comedies                    1674
International TV Shows      1351
Documentaries                869
Action & Adventure           859
TV Dramas                    763
Independent Movies           756
Children & Family Movies     641
Romantic Movies              616
TV Comedies                  581
Thrillers                    577
Crime TV Shows               470
Kids' TV                     451
Docuseries                   395
Name: count, dtype: int64

In [200]:
# Director and cast cleaning
for col in ['director', 'cast']:
    if col in df.columns:
        df[col] = df[col].fillna('').astype('string').str.strip() # Fill NaN and trim whitespace
        df[f'{col}_missing'] = df[col].apply(lambda x: True if (not x or pd.isna(x)) else False) # True if missing or empty
        df[f'{col}_primary'] = df[col].replace('', pd.NA).dropna().str.split(',').str[0].str.strip().fillna('Unknown') # First director/cast or Unknown if missing
    else:
        df[f'{col}_missing'] = True # All missing
        df[f'{col}_primary'] = 'Unknown' # All Unknown

print("Director primary sample:")
display(df[['director', 'director_primary']].head(5)) # Sample of director primary
print("Cast primary sample:")
display(df[['cast', 'cast_primary']].head(5)) # Sample of cast primary

Director primary sample:


,director,director_primary
0,Kirsten Johnson,Kirsten Johnson
1,,NaN
2,Julien Leclercq,Julien Leclercq
3,,NaN
4,,NaN


Cast primary sample:


,cast,cast_primary
0,,NaN
1,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thabang Molaba, Dillon Windvogel, Natasha Thahane, Arno Greeff, Xolile Tshabalala, Getmore Sithole, Cindy Mahlangu, Ryle De Morny, Greteli Fincham, Sello Ma...",Ama Qamata
2,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabiha Akkari, Sofia Lesaffre, Salim Kechiouche, Noureddine Farihi, Geert Van Rampelberg, Bakary Diombera",Sami Bouajila
3,,NaN
4,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam Khan, Ahsaas Channa, Revathi Pillai, Urvi Singh, Arun Kumar",Mayur More


In [201]:
# Numeric and categorical cleanups
if 'release_year' in df.columns:
    df['release_year'] = pd.to_numeric(df['release_year'], errors='coerce').astype('Int64') # Coerce errors to NaN, use Int64 to allow NaN
if 'rating' in df.columns:
    df['rating'] = df['rating'].fillna('Not Rated').astype('string') # Fill NaN and convert to string
if 'type' in df.columns:
    df['type'] = df['type'].astype('string').str.strip().str.title()  # Movie / Tv Show standardization with title case
print("'release_year' data type:", df['release_year'].dtype if 'release_year' in df.columns else 'N/A')

'release_year' data type: Int64


In [202]:
# Build metadata text column (for TF-IDF later)
def safe(x):
    if pd.isna(x):
        return "" # Handle NaN by returning empty string so that it doesn't affect metadata
    if isinstance(x, list):
        return " ".join(x) # Join list into space-separated string
    return str(x)

text_fields = []
if 'title' in df.columns: text_fields.append('title')
if 'description' in df.columns: text_fields.append('description')
# Add engineered/text fields to get more context
text_fields += ['genres_str', 'director_primary', 'cast_primary', 'primary_country']

def combine_metadata(row): # Combine specified text fields into one metadata string
    parts = [safe(row.get(f, "")) for f in text_fields] # Get each field safely from row, default to empty string
    parts = [p for p in parts if p] # Keep only non-empty parts
    return " | ".join(parts) # Join with " | "

df['metadata'] = df.apply(combine_metadata, axis=1) # Apply row-wise to combine metadata
print("Metadata sample:")
display(df[['title','metadata']].head(3))

Metadata sample:


,title,metadata
0,Dick Johnson Is Dead,"Dick Johnson Is Dead | As her father nears the end of his life, filmmaker Kirsten Johnson stages his death in inventive and comical ways to help them both face the inevitable. | Documentaries | Ki..."
1,Blood & Water,"Blood & Water | After crossing paths at a party, a Cape Town teen sets out to prove whether a private-school swimming star is her sister who was abducted at birth. | International TV Shows|TV Dram..."
2,Ganglands,"Ganglands | To protect his family from a powerful drug lord, skilled thief Mehdi and his expert team of robbers are pulled into a violent and deadly turf war. | Crime TV Shows|International TV Sho..."


In [203]:
# Dedupe and sanity checks
before = df.shape[0] # Initial row count
# Drop exact duplicate rows first
df.loc[df.astype(str).drop_duplicates().index] # Converted to string because some columns may be lists and therefore unhashable (drop_duplicates() requires hashable types)
# Drop duplicates by title + type + release_year if many duplicates expected
if set(['title','type','release_year']).issubset(df.columns):
    df = df.drop_duplicates(subset=['title','type','release_year'])
after = df.shape[0] # Final row count
print(f"Rows: Before = {before}, After dedupe = {after}, Removed = {before-after}")

print("\nTop missing counts (post-clean):")
display(df.isnull().sum().sort_values(ascending=False).head(20))

Rows: Before = 8807, After dedupe = 8804, Removed = 3

Top missing counts (post-clean):


director_primary    2633
country              830
cast_primary         825
date_added            10
date_added_month      10
date_added_year       10
duration               3
cast                   0
title                  0
type                   0
show_id                0
director               0
rating                 0
release_year           0
listed_in              0
description            0
primary_country        0
genres_list            0
director_missing       0
genres_str             0
dtype: int64

In [204]:
# Select columns to save and export cleaned csv
CLEAN_DIR.mkdir(parents=True, exist_ok=True) # Ensure clean directory exists and create if needed

cols = []
# Prefer canonical ids if present
if 'show_id' in df.columns: cols.append('show_id')
# Core fields
cols += [c for c in ['type','title','release_year','primary_country','genres_str','rating','date_added','date_added_year','date_added_month','director_primary','cast_primary','metadata'] if c in df.columns]
# Relevance of above fields: They cover the main attributes needed for analysis and recommendation, while avoiding overly detailed or redundant info

# Ensure title is first if present
if 'title' in cols:
    cols = ['title'] + [c for c in cols if c!='title']

df_clean = df[cols].copy() # Create a copy of the selected columns to avoid SettingWithCopyWarning
# SettingWithCopyWarning can occur if we try to modify df_clean later without using .copy()
print("Saving cleaned CSV to:", CLEAN_CSV)
df_clean.to_csv(CLEAN_CSV, index=False) # Save without index
print("Saved. \nClean shape:", df_clean.shape)
display(df_clean.head(4))

Saving cleaned CSV to: C:\Users\Admibn\OneDrive\Desktop\Netflix Project\netflix-analysis-project\data\cleaned\netflix_titles_clean.csv
Saved. 
Clean shape: (8804, 13)


,title,show_id,type,release_year,primary_country,genres_str,rating,date_added,date_added_year,date_added_month,director_primary,cast_primary,metadata
0,Dick Johnson Is Dead,s1,Movie,2020,United States,Documentaries,PG-13,2021-09-25,2021,9,Kirsten Johnson,NaN,"Dick Johnson Is Dead | As her father nears the end of his life, filmmaker Kirsten Johnson stages his death in inventive and comical ways to help them both face the inevitable. | Documentaries | Ki..."
1,Blood & Water,s2,Tv Show,2021,South Africa,International TV Shows|TV Dramas|TV Mysteries,TV-MA,2021-09-24,2021,9,NaN,Ama Qamata,"Blood & Water | After crossing paths at a party, a Cape Town teen sets out to prove whether a private-school swimming star is her sister who was abducted at birth. | International TV Shows|TV Dram..."
2,Ganglands,s3,Tv Show,2021,Unknown,Crime TV Shows|International TV Shows|TV Action & Adventure,TV-MA,2021-09-24,2021,9,Julien Leclercq,Sami Bouajila,"Ganglands | To protect his family from a powerful drug lord, skilled thief Mehdi and his expert team of robbers are pulled into a violent and deadly turf war. | Crime TV Shows|International TV Sho..."
3,Jailbirds New Orleans,s4,Tv Show,2021,Unknown,Docuseries|Reality TV,TV-MA,2021-09-24,2021,9,NaN,NaN,"Jailbirds New Orleans | Feuds, flirtations and toilet talk go down among the incarcerated women at the Orleans Justice Center in New Orleans on this gritty reality series. | Docuseries|Reality TV ..."


In [205]:
# Quick summary counts to include in the notebook
print("Unique titles:", df_clean['title'].nunique()) # Number of unique titles
if 'primary_country' in df_clean.columns:
    print("Top 8 countries:")
    display(df_clean['primary_country'].value_counts().head(8))
if 'genres_str' in df_clean.columns:
    print("Top 8 genres (by primary_genre approximation):")
    display(df_clean['genres_str'].str.split('|').str[0].value_counts().head(8)) # Approximate primary genre by first listed genre

Unique titles: 8801
Top 8 countries:


primary_country
United States     3211
India             1007
Unknown            832
United Kingdom     628
Canada             271
Japan              259
France             212
South Korea        211
Name: count, dtype: Int64

Top 8 genres (by primary_genre approximation):


genres_str
Dramas                      1599
Comedies                    1209
Action & Adventure           859
Documentaries                829
International TV Shows       773
Children & Family Movies     605
Crime TV Shows               399
Kids' TV                     388
Name: count, dtype: int64